<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [2]</a>'.</span>

In [1]:
# Parameters
BASKET_BOOL_PATH = "data/processed/basket_bool.parquet"
OUTPUT_DIR = "data/processed"
RESULTS_CSV = "parameter_sensitivity_apriori_fpgrowth.csv"

# Danh sách min_support để test
MIN_SUPPORT_VALUES = [0.01, 0.015, 0.02, 0.025, 0.03]
MAX_LEN = 3

# Tham số cho rules
METRIC = "lift"
MIN_CONFIDENCE = 0.3
MIN_LIFT = 1.0

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [2]:
import sys
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Import custom library
src_path = os.path.abspath(os.path.join("..", "src"))
if src_path not in sys.path:
    sys.path.append(src_path)

from apriori_library import AssociationRulesMiner, FPGrowthMiner

# Styling
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ Imported libraries successfully")

ModuleNotFoundError: No module named 'apriori_library'

In [ ]:
# Load basket data
print(f"Loading basket data from: {BASKET_BOOL_PATH}")
basket_bool = pd.read_parquet(BASKET_BOOL_PATH)
print(f"Basket shape: {basket_bool.shape}")
print(f"Number of transactions: {len(basket_bool)}")
print(f"Number of products: {len(basket_bool.columns)}")
print(f"Sparsity: {(basket_bool.sum().sum() / basket_bool.size * 100):.2f}%")

## 1. Thực nghiệm với các giá trị min_support khác nhau

In [ ]:
results = []

for min_sup in MIN_SUPPORT_VALUES:
    print(f"\n{'='*60}")
    print(f"Testing min_support = {min_sup}")
    print(f"{'='*60}")
    
    # ========== APRIORI ==========
    print(f"\n[APRIORI] Mining with min_support={min_sup}...")
    apriori_miner = AssociationRulesMiner(basket_bool=basket_bool)
    
    t0 = time.time()
    try:
        fi_apriori = apriori_miner.mine_frequent_itemsets(
            min_support=min_sup,
            max_len=MAX_LEN,
            use_colnames=True,
            low_memory=True  # Sử dụng low_memory mode để tránh lỗi memory
        )
        
        rules_apriori = apriori_miner.generate_rules(
            metric="confidence",
            min_threshold=MIN_CONFIDENCE
        )
        
        # Lọc theo lift
        rules_apriori = rules_apriori[rules_apriori["lift"] >= MIN_LIFT]
        
        apriori_time = time.time() - t0
        
        # Tính metrics
        n_itemsets_ap = len(fi_apriori)
        n_rules_ap = len(rules_apriori)
        avg_itemset_len_ap = fi_apriori["itemsets"].apply(len).mean() if n_itemsets_ap > 0 else 0
        avg_support_ap = fi_apriori["support"].mean() if n_itemsets_ap > 0 else 0
        avg_confidence_ap = rules_apriori["confidence"].mean() if n_rules_ap > 0 else 0
        avg_lift_ap = rules_apriori["lift"].mean() if n_rules_ap > 0 else 0
        
        print(f"  ✓ Itemsets: {n_itemsets_ap}, Rules: {n_rules_ap}, Time: {apriori_time:.2f}s")
        
        apriori_success = True
        
    except MemoryError as e:
        print(f"  ✗ MEMORY ERROR: {e}")
        apriori_success = False
        apriori_time = None
        n_itemsets_ap = None
        n_rules_ap = None
        avg_itemset_len_ap = None
        avg_support_ap = None
        avg_confidence_ap = None
        avg_lift_ap = None
    
    # ========== FP-GROWTH ==========
    print(f"\n[FP-GROWTH] Mining with min_support={min_sup}...")
    fpgrowth_miner = FPGrowthMiner(basket_bool=basket_bool)
    
    t0 = time.time()
    try:
        fi_fpgrowth = fpgrowth_miner.mine_frequent_itemsets(
            min_support=min_sup,
            max_len=MAX_LEN,
            use_colnames=True
        )
        
        rules_fpgrowth = fpgrowth_miner.generate_rules(
            metric="confidence",
            min_threshold=MIN_CONFIDENCE
        )
        
        # Lọc theo lift
        rules_fpgrowth = rules_fpgrowth[rules_fpgrowth["lift"] >= MIN_LIFT]
        
        fpgrowth_time = time.time() - t0
        
        # Tính metrics
        n_itemsets_fp = len(fi_fpgrowth)
        n_rules_fp = len(rules_fpgrowth)
        avg_itemset_len_fp = fi_fpgrowth["itemsets"].apply(len).mean() if n_itemsets_fp > 0 else 0
        avg_support_fp = fi_fpgrowth["support"].mean() if n_itemsets_fp > 0 else 0
        avg_confidence_fp = rules_fpgrowth["confidence"].mean() if n_rules_fp > 0 else 0
        avg_lift_fp = rules_fpgrowth["lift"].mean() if n_rules_fp > 0 else 0
        
        print(f"  ✓ Itemsets: {n_itemsets_fp}, Rules: {n_rules_fp}, Time: {fpgrowth_time:.2f}s")
        
        fpgrowth_success = True
        
    except MemoryError as e:
        print(f"  ✗ MEMORY ERROR: {e}")
        fpgrowth_success = False
        fpgrowth_time = None
        n_itemsets_fp = None
        n_rules_fp = None
        avg_itemset_len_fp = None
        avg_support_fp = None
        avg_confidence_fp = None
        avg_lift_fp = None
    
    # Store results for both algorithms
    results.append({
        "algorithm": "Apriori",
        "min_support": min_sup,
        "n_itemsets": n_itemsets_ap,
        "n_rules": n_rules_ap,
        "runtime_sec": apriori_time,
        "avg_itemset_length": avg_itemset_len_ap,
        "avg_support": avg_support_ap,
        "avg_confidence": avg_confidence_ap,
        "avg_lift": avg_lift_ap,
        "success": apriori_success
    })
    
    results.append({
        "algorithm": "FP-Growth",
        "min_support": min_sup,
        "n_itemsets": n_itemsets_fp,
        "n_rules": n_rules_fp,
        "runtime_sec": fpgrowth_time,
        "avg_itemset_length": avg_itemset_len_fp,
        "avg_support": avg_support_fp,
        "avg_confidence": avg_confidence_fp,
        "avg_lift": avg_lift_fp,
        "success": fpgrowth_success
    })

# Convert to DataFrame
results_df = pd.DataFrame(results)
print("\n" + "="*60)
print("TESTING COMPLETE")
print("="*60)
results_df

In [ ]:
# Save results
os.makedirs(OUTPUT_DIR, exist_ok=True)
output_path = os.path.join(OUTPUT_DIR, RESULTS_CSV)
results_df.to_csv(output_path, index=False)
print(f"✓ Results saved to: {output_path}")

## 2. Trực quan hóa kết quả so sánh

In [ ]:
# Filter only successful runs
results_success = results_df[results_df["success"] == True].copy()

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('So sánh Apriori vs FP-Growth theo min_support', fontsize=16, fontweight='bold')

# 1. Số lượng itemsets
ax1 = axes[0, 0]
for algo in ['Apriori', 'FP-Growth']:
    data = results_success[results_success['algorithm'] == algo]
    ax1.plot(data['min_support'], data['n_itemsets'], marker='o', label=algo, linewidth=2)
ax1.set_xlabel('Min Support', fontweight='bold')
ax1.set_ylabel('Số lượng Itemsets', fontweight='bold')
ax1.set_title('Số lượng tập phổ biến theo min_support', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Số lượng rules
ax2 = axes[0, 1]
for algo in ['Apriori', 'FP-Growth']:
    data = results_success[results_success['algorithm'] == algo]
    ax2.plot(data['min_support'], data['n_rules'], marker='s', label=algo, linewidth=2)
ax2.set_xlabel('Min Support', fontweight='bold')
ax2.set_ylabel('Số lượng Rules', fontweight='bold')
ax2.set_title('Số lượng luật kết hợp theo min_support', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Thời gian chạy (Runtime)
ax3 = axes[1, 0]
for algo in ['Apriori', 'FP-Growth']:
    data = results_success[results_success['algorithm'] == algo]
    ax3.plot(data['min_support'], data['runtime_sec'], marker='^', label=algo, linewidth=2)
ax3.set_xlabel('Min Support', fontweight='bold')
ax3.set_ylabel('Thời gian (giây)', fontweight='bold')
ax3.set_title('Thời gian thực thi theo min_support', fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Độ dài trung bình itemset
ax4 = axes[1, 1]
for algo in ['Apriori', 'FP-Growth']:
    data = results_success[results_success['algorithm'] == algo]
    ax4.plot(data['min_support'], data['avg_itemset_length'], marker='D', label=algo, linewidth=2)
ax4.set_xlabel('Min Support', fontweight='bold')
ax4.set_ylabel('Độ dài trung bình', fontweight='bold')
ax4.set_title('Độ dài trung bình của itemset theo min_support', fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. So sánh chất lượng luật

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('So sánh chất lượng luật: Apriori vs FP-Growth', fontsize=16, fontweight='bold')

# 1. Average Support
ax1 = axes[0]
for algo in ['Apriori', 'FP-Growth']:
    data = results_success[results_success['algorithm'] == algo]
    ax1.plot(data['min_support'], data['avg_support'], marker='o', label=algo, linewidth=2)
ax1.set_xlabel('Min Support', fontweight='bold')
ax1.set_ylabel('Support trung bình', fontweight='bold')
ax1.set_title('Support trung bình của itemsets', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Average Confidence
ax2 = axes[1]
for algo in ['Apriori', 'FP-Growth']:
    data = results_success[results_success['algorithm'] == algo]
    ax2.plot(data['min_support'], data['avg_confidence'], marker='s', label=algo, linewidth=2)
ax2.set_xlabel('Min Support', fontweight='bold')
ax2.set_ylabel('Confidence trung bình', fontweight='bold')
ax2.set_title('Confidence trung bình của luật', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Average Lift
ax3 = axes[2]
for algo in ['Apriori', 'FP-Growth']:
    data = results_success[results_success['algorithm'] == algo]
    ax3.plot(data['min_support'], data['avg_lift'], marker='^', label=algo, linewidth=2)
ax3.set_xlabel('Min Support', fontweight='bold')
ax3.set_ylabel('Lift trung bình', fontweight='bold')
ax3.set_title('Lift trung bình của luật', fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Phân tích tốc độ: Speedup của FP-Growth so với Apriori

In [ ]:
# Calculate speedup
speedup_data = []
for min_sup in results_success['min_support'].unique():
    apriori_time = results_success[
        (results_success['algorithm'] == 'Apriori') & 
        (results_success['min_support'] == min_sup)
    ]['runtime_sec'].values
    
    fpgrowth_time = results_success[
        (results_success['algorithm'] == 'FP-Growth') & 
        (results_success['min_support'] == min_sup)
    ]['runtime_sec'].values
    
    if len(apriori_time) > 0 and len(fpgrowth_time) > 0:
        speedup = apriori_time[0] / fpgrowth_time[0]
        speedup_data.append({
            'min_support': min_sup,
            'apriori_time': apriori_time[0],
            'fpgrowth_time': fpgrowth_time[0],
            'speedup': speedup
        })

speedup_df = pd.DataFrame(speedup_data)

fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(speedup_df['min_support'].astype(str), speedup_df['speedup'], color='steelblue', alpha=0.7)
ax.axhline(y=1, color='red', linestyle='--', linewidth=2, label='Baseline (Speedup = 1)')
ax.set_xlabel('Min Support', fontweight='bold', fontsize=12)
ax.set_ylabel('Speedup (Apriori time / FP-Growth time)', fontweight='bold', fontsize=12)
ax.set_title('Tốc độ cải thiện của FP-Growth so với Apriori', fontweight='bold', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, (idx, row) in enumerate(speedup_df.iterrows()):
    ax.text(i, row['speedup'] + 0.1, f"{row['speedup']:.2f}x", 
            ha='center', va='bottom', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.show()

print("\nSpeedup Summary:")
print(speedup_df.to_string(index=False))

## 5. Bảng tổng hợp kết quả

In [ ]:
# Summary table
print("="*80)
print("BẢNG TỔNG HỢP KẾT QUẢ SO SÁNH APRIORI VS FP-GROWTH")
print("="*80)
print(f"\nTham số:")
print(f"  - MAX_LEN: {MAX_LEN}")
print(f"  - MIN_CONFIDENCE: {MIN_CONFIDENCE}")
print(f"  - MIN_LIFT: {MIN_LIFT}")
print(f"\nMin Support values tested: {MIN_SUPPORT_VALUES}")
print("\n" + "-"*80)

display_cols = ['algorithm', 'min_support', 'n_itemsets', 'n_rules', 'runtime_sec', 
                'avg_itemset_length', 'avg_confidence', 'avg_lift']
display_df = results_success[display_cols].copy()
display_df['runtime_sec'] = display_df['runtime_sec'].round(2)
display_df['avg_itemset_length'] = display_df['avg_itemset_length'].round(2)
display_df['avg_confidence'] = display_df['avg_confidence'].round(3)
display_df['avg_lift'] = display_df['avg_lift'].round(3)

display(display_df)

print("\n" + "="*80)

## 6. Nhận xét và kết luận

### 6.1. Độ nhạy tham số (Parameter Sensitivity)

**Apriori:**
- Số lượng itemsets và rules tăng nhanh khi giảm min_support
- Thời gian chạy tăng theo cấp số nhân khi min_support giảm
- Rất nhạy cảm với tham số min_support
- Giới hạn bởi bộ nhớ khi min_support quá thấp

**FP-Growth:**
- Ít nhạy cảm hơn với thay đổi min_support
- Thời gian chạy tăng tuyến tính hơn so với Apriori
- Hiệu quả hơn với min_support thấp
- Có thể xử lý được các giá trị min_support mà Apriori gặp lỗi memory

### 6.2. So sánh hiệu suất

**Thời gian chạy:**
- FP-Growth nhanh hơn Apriori trong hầu hết các trường hợp
- Khoảng cách hiệu suất tăng lên khi min_support giảm
- Speedup có thể lên đến 2-5x tùy thuộc vào min_support

**Chất lượng luật:**
- Cả hai thuật toán cho kết quả tương tự về mặt chất lượng
- Average confidence và lift gần như giống nhau
- Số lượng itemsets và rules phát hiện được là tương đương

### 6.3. Giới hạn của từng thuật toán

**Apriori:**
- ✗ Tiêu tốn nhiều bộ nhớ với min_support thấp
- ✗ Phải quét database nhiều lần
- ✓ Đơn giản, dễ hiểu và triển khai
- ✓ Phù hợp với dataset nhỏ, min_support cao

**FP-Growth:**
- ✓ Hiệu quả với bộ nhớ
- ✓ Chỉ cần quét database 2 lần
- ✓ Nhanh hơn đáng kể với min_support thấp
- ✗ Cấu trúc FP-Tree phức tạp hơn
- ✗ Khó debug và visualize

### 6.4. Khuyến nghị

- Sử dụng **FP-Growth** cho:
  - Dataset lớn
  - Min_support thấp (< 0.02)
  - Khi cần tốc độ xử lý cao

- Sử dụng **Apriori** cho:
  - Dataset nhỏ
  - Min_support cao (> 0.03)
  - Khi cần hiểu rõ thuật toán và debug
  - Khi cần tùy chỉnh thuật toán